<a href="https://colab.research.google.com/github/profliuhao/CSIT599/blob/main/CSIT599_Module2_In_Class_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSIT 599 - Module 2 In Class Exercise



## Cats vs Dogs Classification using Convolutional Neural Networks (CNN)

### Exercise for Students

This exercise demonstrates the power of CNNs for image classification tasks.

Downloads dataset to current working directory to avoid permission issues.

You'll learn about:
- Image preprocessing and data augmentation
- Convolutional layers and feature maps
- Pooling layers and dimensionality reduction
- CNN architecture design
- Training with callbacks and regularization

Instructions:
1. Fill in the blanks marked with "# TODO: STUDENT FILL IN"
2. Run the code and observe the CNN's performance
3. Experiment with different architectures and hyperparameters

In [1]:
"""
Dataset: Automatically downloads cats vs dogs sample dataset to current directory
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import urllib.request
import zipfile

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
GPU Available: []


In [ ]:
# ============================================================================
# PART 1: LOCAL DATA DOWNLOAD AND PREPROCESSING
# ============================================================================

def download_cats_dogs_dataset():
    """
    Download cats vs dogs dataset to current working directory.
    This avoids permission issues with system directories.

    Returns:
        tuple: (train_dir, validation_dir) paths
    """
    print("Downloading cats vs dogs dataset to current directory...")

    # Use current working directory - no permission issues!
    current_dir = os.getcwd()
    dataset_dir = os.path.join(current_dir, "cats_dogs_dataset")
    zip_path = os.path.join(current_dir, "cats_and_dogs.zip")

    # Dataset URL
    dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"

    print(f"Working directory: {current_dir}")
    print(f"Dataset will be saved to: {dataset_dir}")

    # Check if dataset already exists
    if os.path.exists(dataset_dir):
        print("Dataset already exists locally!")
        train_dir = os.path.join(dataset_dir, "train")
        validation_dir = os.path.join(dataset_dir, "validation")

        if verify_dataset_structure(train_dir, validation_dir):
            return train_dir, validation_dir
        else:
            print("Existing dataset is incomplete. Re-downloading...")
            import shutil
            shutil.rmtree(dataset_dir)

    try:
        # Download the zip file
        print(f"Downloading from: {dataset_url}")
        print("This may take a few minutes...")

        def progress_hook(block_num, block_size, total_size):
            downloaded = block_num * block_size
            if total_size > 0:
                percent = min(100, (downloaded * 100) / total_size)
                mb_downloaded = downloaded / (1024 * 1024)
                mb_total = total_size / (1024 * 1024)
                print(f"\rProgress: {percent:.1f}% ({mb_downloaded:.1f}/{mb_total:.1f} MB)",
                      end='', flush=True)

        urllib.request.urlretrieve(dataset_url, zip_path, reporthook=progress_hook)
        print(f"\nDownload completed: {zip_path}")

        # Extract the zip file
        print("Extracting dataset...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(current_dir)

        # Find the extracted directory
        extracted_dir = None
        for item in os.listdir(current_dir):
            if os.path.isdir(item) and "cats_and_dogs" in item.lower():
                extracted_dir = os.path.join(current_dir, item)
                break

        if extracted_dir:
            # Rename to standard name
            if extracted_dir != dataset_dir:
                os.rename(extracted_dir, dataset_dir)
        else:
            raise Exception("Could not find extracted dataset directory")

        # Clean up zip file
        if os.path.exists(zip_path):
            os.remove(zip_path)
            print("Cleaned up zip file")

        # Set up directory paths
        train_dir = os.path.join(dataset_dir, "train")
        validation_dir = os.path.join(dataset_dir, "validation")

        # Verify structure
        if verify_dataset_structure(train_dir, validation_dir):
            print("Dataset setup completed successfully!")
            return train_dir, validation_dir
        else:
            raise Exception("Dataset structure verification failed")

    except Exception as e:
        print(f"Download failed: {str(e)}")
        print("Creating minimal demo dataset...")
        return create_minimal_demo_dataset()

def verify_dataset_structure(train_dir, validation_dir):
    """
    Verify that the dataset has the expected structure and content.

    Args:
        train_dir: Training directory path
        validation_dir: Validation directory path

    Returns:
        bool: True if structure is valid
    """
    required_dirs = [
        os.path.join(train_dir, 'cats'),
        os.path.join(train_dir, 'dogs'),
        os.path.join(validation_dir, 'cats'),
        os.path.join(validation_dir, 'dogs')
    ]

    print("\nVerifying dataset structure...")
    for dir_path in required_dirs:
        if not os.path.exists(dir_path):
            print(f"Missing directory: {dir_path}")
            return False

        # Count image files
        image_files = [f for f in os.listdir(dir_path)
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        print(f"{len(image_files)} images in {os.path.basename(dir_path)}")

        if len(image_files) == 0:
            print(f"No images found in: {dir_path}")
            return False

    return True

def create_minimal_demo_dataset():
    """
    Create a minimal dataset with synthetic images for demonstration.
    This runs if the download fails for any reason.

    Returns:
        tuple: (train_dir, validation_dir)
    """
    print("Creating minimal demo dataset with synthetic images...")

    try:
        from PIL import Image
        import numpy as np

        # Create directory structure in current directory
        dataset_dir = os.path.join(os.getcwd(), "demo_cats_dogs")
        train_dir = os.path.join(dataset_dir, "train")
        validation_dir = os.path.join(dataset_dir, "validation")

        for split in ['train', 'validation']:
            for category in ['cats', 'dogs']:
                os.makedirs(os.path.join(dataset_dir, split, category), exist_ok=True)

        # Create synthetic images with distinctive patterns
        def create_synthetic_image(category, size=(150, 150)):
            """Create a synthetic image with distinctive patterns for each class."""
            image = np.random.randint(50, 200, (size[0], size[1], 3), dtype=np.uint8)

            if category == 'cats':
                # Add horizontal stripes for cats
                for i in range(0, size[0], 20):
                    image[i:i+5, :, :] = [255, 200, 100]  # Orange stripes
            else:  # dogs
                # Add diagonal pattern for dogs
                for i in range(size[0]):
                    for j in range(size[1]):
                        if (i + j) % 30 < 10:
                            image[i, j, :] = [100, 150, 255]  # Blue diagonal

            return Image.fromarray(image)

        # Generate training images
        print("Generating training images...")
        for category in ['cats', 'dogs']:
            category_dir = os.path.join(train_dir, category)
            for i in range(50):  # 50 images per category
                img = create_synthetic_image(category)
                img.save(os.path.join(category_dir, f'{category}_{i:03d}.jpg'))

        # Generate validation images
        print("Generating validation images...")
        for category in ['cats', 'dogs']:
            category_dir = os.path.join(validation_dir, category)
            for i in range(10):  # 10 images per category
                img = create_synthetic_image(category)
                img.save(os.path.join(category_dir, f'{category}_val_{i:03d}.jpg'))

        print(f"Demo dataset created at: {dataset_dir}")
        print("NOTE: This is synthetic data for demonstration only!")

        return train_dir, validation_dir

    except ImportError:
        print("PIL (Pillow) not available. Install with: pip install pillow")
        raise
    except Exception as e:
        print(f"Failed to create demo dataset: {str(e)}")
        raise

def create_data_generators(train_dir, validation_dir, img_height=150, img_width=150, batch_size=32):
    """
    Create data generators for training and validation with data augmentation.

    Data augmentation helps prevent overfitting by creating variations of training images.
    This is crucial for small datasets and improves model generalization.

    Args:
        train_dir: Path to training data directory
        validation_dir: Path to validation data directory
        img_height: Target height for resized images
        img_width: Target width for resized images
        batch_size: Number of images per batch

    Returns:
        tuple: (train_generator, validation_generator)
    """
    print(f"\nCreating data generators...")
    print(f"Training directory: {train_dir}")
    print(f"Validation directory: {validation_dir}")
    print(f"Image size: {img_height}x{img_width}")
    print(f"Batch size: {batch_size}")

    # TODO: STUDENT FILL IN
    # Create ImageDataGenerator for training data with augmentation
    # Include: rescale=1./255 (normalize), rotation_range=20, width_shift_range=0.2,
    # height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True
    train_datagen = ImageDataGenerator(
        rescale=________,           # Normalize pixel values to [0,1]
        rotation_range=________,    # Randomly rotate images up to 20 degrees
        width_shift_range=________,  # Randomly shift images horizontally by 20%
        height_shift_range=________, # Randomly shift images vertically by 20%
        shear_range=________,       # Randomly apply shear transformations
        zoom_range=________,        # Randomly zoom in/out by 20%
        horizontal_flip=________,   # Randomly flip images horizontally
        fill_mode='nearest'         # Fill in newly created pixels
    )

    # TODO: STUDENT FILL IN
    # Create ImageDataGenerator for validation data (only rescaling, no augmentation)
    # We don't augment validation data because we want consistent evaluation
    validation_datagen = ImageDataGenerator(rescale=________)

    # TODO: STUDENT FILL IN
    # Create training data generator
    # Use flow_from_directory with target_size=(img_height, img_width),
    # batch_size=batch_size, class_mode='binary' (for cats vs dogs)
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(________, ________),
        batch_size=________,
        class_mode='________'  # 'binary' for 2 classes (cats vs dogs)
    )

    # TODO: STUDENT FILL IN
    # Create validation data generator (similar to training but with validation_datagen and validation_dir)
    validation_generator = validation_datagen.flow_from_directory(
        ________,  # validation_dir
        target_size=(________, ________),
        batch_size=________,
        class_mode='________'
    )

    print(f"Classes found: {train_generator.class_indices}")
    print(f"Training samples: {train_generator.samples}")
    print(f"Validation samples: {validation_generator.samples}")

    return train_generator, validation_generator

def visualize_sample_images(train_generator):
    """
    Display a batch of training images to understand the data.
    This helps verify that data loading and augmentation are working correctly.

    Args:
        train_generator: Training data generator
    """
    print("Displaying sample images...")

    # Get a batch of images and labels
    sample_batch = next(train_generator)
    images, labels = sample_batch

    # Plot first 8 images
    plt.figure(figsize=(12, 8))
    for i in range(min(8, len(images))):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        # Convert label to class name (0=cats, 1=dogs by default)
        label_name = 'Dog' if labels[i] == 1 else 'Cat'
        plt.title(f'Label: {label_name}')
        plt.axis('off')

    plt.suptitle('Sample Training Images (with augmentation)')
    plt.tight_layout()
    plt.show()



In [ ]:
# ============================================================================
# PART 2: CONVOLUTIONAL NEURAL NETWORK ARCHITECTURE
# ============================================================================

def create_simple_cnn(img_height=150, img_width=150):
    """
    Create a simple CNN model for binary classification.

    CNN Architecture Explanation:
    1. Convolutional layers extract features (edges, shapes, textures)
       - Early layers detect simple features (edges, corners)
       - Deeper layers detect complex features (shapes, objects)
    2. Pooling layers reduce spatial dimensions and computation
       - MaxPooling keeps the strongest activation in each region
    3. Dense layers perform final classification based on extracted features

    Args:
        img_height: Input image height
        img_width: Input image width

    Returns:
        keras.Model: Compiled CNN model
    """
    print("\n" + "="*50)
    print("CREATING SIMPLE CNN MODEL")
    print("="*50)

    model = keras.Sequential(name="Simple_CNN")

    # TODO: STUDENT FILL IN
    # First Convolutional Block
    # Add Conv2D layer: 32 filters, (3,3) kernel, 'relu' activation
    # Specify input_shape=(img_height, img_width, 3) for RGB images
    model.add(layers.Conv2D(
        filters=________,
        kernel_size=(________, ________),
        activation='________',
        input_shape=(________, ________, ________),  # (height, width, channels)
        name='conv2d_1'
    ))

    # TODO: STUDENT FILL IN
    # Add MaxPooling2D layer with (2,2) pool size to reduce spatial dimensions
    model.add(layers.MaxPooling2D(pool_size=(________, ________), name='maxpool_1'))

    # TODO: STUDENT FILL IN
    # Second Convolutional Block
    # Add Conv2D layer: 64 filters, (3,3) kernel, 'relu' activation
    # More filters allow learning more complex features
    model.add(layers.Conv2D(
        filters=________,
        kernel_size=(________, ________),
        activation='________',
        name='conv2d_2'
    ))
    model.add(layers.MaxPooling2D(pool_size=(________, ________), name='maxpool_2'))

    # TODO: STUDENT FILL IN
    # Third Convolutional Block
    # Add Conv2D layer: 128 filters, (3,3) kernel, 'relu' activation
    # Even more filters for abstract/high-level features
    model.add(layers.Conv2D(
        filters=________,
        kernel_size=(________, ________),
        activation='________',
        name='conv2d_3'
    ))
    model.add(layers.MaxPooling2D(pool_size=(________, ________), name='maxpool_3'))

    # TODO: STUDENT FILL IN
    # Flatten the 3D feature maps to 1D for dense layers
    # This converts from spatial features to a vector
    model.add(layers.Flatten(name='flatten'))

    # TODO: STUDENT FILL IN
    # Add Dense layer with 512 units and 'relu' activation
    # This learns combinations of the extracted features
    model.add(layers.Dense(units=________, activation='________', name='dense_1'))

    # TODO: STUDENT FILL IN
    # Add Dropout layer with 0.5 rate to prevent overfitting
    # Randomly sets 50% of neurons to 0 during training
    model.add(layers.Dropout(rate=________, name='dropout'))

    # TODO: STUDENT FILL IN
    # Output layer: 1 unit with 'sigmoid' activation for binary classification
    # Sigmoid outputs probability between 0 and 1
    model.add(layers.Dense(units=________, activation='________', name='output'))

    # TODO: STUDENT FILL IN
    # Compile model: optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']
    # Adam: adaptive learning rate optimizer
    # Binary crossentropy: loss function for binary classification
    model.compile(
        optimizer='________',
        loss='________',
        metrics=['________']
    )

    print("\nSimple CNN Architecture:")
    model.summary()

    return model

def create_improved_cnn(img_height=150, img_width=150):
    """
    Create an improved CNN model with batch normalization and more layers.

    Improvements over simple CNN:
    - Batch normalization for better training stability
    - More convolutional layers for better feature extraction
    - Additional regularization techniques
    - Deeper architecture for more complex pattern recognition

    Args:
        img_height: Input image height
        img_width: Input image width

    Returns:
        keras.Model: Compiled improved CNN model
    """
    print("\n" + "="*50)
    print("CREATING IMPROVED CNN MODEL")
    print("="*50)

    model = keras.Sequential(name="Improved_CNN")

    # First Block: Basic feature detection
    model.add(layers.Conv2D(32, (3, 3), activation='relu',
                           input_shape=(img_height, img_width, 3), name='conv2d_1'))
    model.add(layers.BatchNormalization(name='bn_1'))  # Normalize activations
    model.add(layers.MaxPooling2D((2, 2), name='maxpool_1'))

    # Second Block: More complex features
    model.add(layers.Conv2D(64, (3, 3), activation='relu', name='conv2d_2'))
    model.add(layers.BatchNormalization(name='bn_2'))
    model.add(layers.MaxPooling2D((2, 2), name='maxpool_2'))

    # Third Block: High-level features
    model.add(layers.Conv2D(128, (3, 3), activation='relu', name='conv2d_3'))
    model.add(layers.BatchNormalization(name='bn_3'))
    model.add(layers.MaxPooling2D((2, 2), name='maxpool_3'))

    # Fourth Block: Abstract features
    model.add(layers.Conv2D(256, (3, 3), activation='relu', name='conv2d_4'))
    model.add(layers.BatchNormalization(name='bn_4'))
    model.add(layers.MaxPooling2D((2, 2), name='maxpool_4'))

    # Dense layers for classification
    model.add(layers.Flatten(name='flatten'))
    model.add(layers.Dense(512, activation='relu', name='dense_1'))
    model.add(layers.Dropout(0.5, name='dropout_1'))
    model.add(layers.Dense(256, activation='relu', name='dense_2'))
    model.add(layers.Dropout(0.3, name='dropout_2'))
    model.add(layers.Dense(1, activation='sigmoid', name='output'))

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print("\nImproved CNN Architecture:")
    model.summary()

    return model



In [ ]:
# ============================================================================
# PART 3: TRAINING AND EVALUATION
# ============================================================================

def train_model(model, train_generator, validation_generator, epochs=10):
    """
    Train the CNN model with early stopping and learning rate reduction.

    Callbacks help improve training:
    - EarlyStopping: Prevents overfitting by stopping when validation loss stops improving
    - ReduceLROnPlateau: Reduces learning rate when training plateaus

    Args:
        model: Keras model to train
        train_generator: Training data generator
        validation_generator: Validation data generator
        epochs: Maximum number of epochs

    Returns:
        keras.callbacks.History: Training history
    """
    print(f"\nTraining {model.name}...")
    print(f"Max epochs: {epochs}")

    # TODO: STUDENT FILL IN
    # Define callbacks for better training
    # EarlyStopping: monitor='val_loss', patience=3, restore_best_weights=True
    # ReduceLROnPlateau: monitor='val_loss', factor=0.2, patience=2
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='________',        # Monitor validation loss
            patience=________,         # Wait 3 epochs before stopping
            restore_best_weights=________  # Restore best weights found
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='________',        # Monitor validation loss
            factor=________,           # Reduce learning rate by factor of 5 (0.2)
            patience=________          # Wait 2 epochs before reducing
        )
    ]

    # Calculate steps per epoch
    # This determines how many batches to process per epoch
    steps_per_epoch = max(1, train_generator.samples // train_generator.batch_size)
    validation_steps = max(1, validation_generator.samples // validation_generator.batch_size)

    print(f"Steps per epoch: {steps_per_epoch}")
    print(f"Validation steps: {validation_steps}")

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps,
        callbacks=callbacks,
        verbose=1  # Show progress bar
    )

    return history

def evaluate_model(model, validation_generator):
    """
    Evaluate model performance and create predictions.

    Args:
        model: Trained Keras model
        validation_generator: Validation data generator

    Returns:
        tuple: (test_accuracy, predictions, true_labels)
    """
    print(f"\n" + "="*50)
    print(f"EVALUATING {model.name.upper()}")
    print("="*50)

    # Reset generator to start from beginning
    validation_generator.reset()

    # Get test accuracy
    test_loss, test_accuracy = model.evaluate(validation_generator, verbose=0)
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

    # Get predictions for detailed analysis
    validation_generator.reset()
    predictions = model.predict(validation_generator, verbose=0)

    # Get true labels
    true_labels = validation_generator.classes

    # Convert predictions to binary (0 or 1)
    pred_labels = (predictions > 0.5).astype(int).flatten()

    # Classification report
    class_names = ['Cat', 'Dog']
    print(f"\nClassification Report for {model.name}:")
    print(classification_report(true_labels, pred_labels, target_names=class_names))

    return test_accuracy, pred_labels, true_labels

def plot_training_history(history, model_name):
    """
    Plot training history to visualize learning progress.

    This helps identify:
    - Overfitting (training accuracy >> validation accuracy)
    - Underfitting (both accuracies are low)
    - Good fit (training and validation curves are close)

    Args:
        history: Training history object
        model_name: Name of the model for title
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Training and validation accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
    ax1.set_title(f'{model_name} - Training vs Validation Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    ax1.grid(True)

    # Training and validation loss
    ax2.plot(history.history['loss'], label='Training Loss', marker='o')
    ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
    ax2.set_title(f'{model_name} - Training vs Validation Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    ax2.grid(True)

    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(y_true, y_pred, model_name):
    """
    Plot confusion matrix for binary classification.

    Confusion matrix shows:
    - True Positives (TP): Correctly predicted dogs
    - True Negatives (TN): Correctly predicted cats
    - False Positives (FP): Incorrectly predicted dogs (actually cats)
    - False Negatives (FN): Incorrectly predicted cats (actually dogs)

    Args:
        y_true: True labels
        y_pred: Predicted labels
        model_name: Name of the model for title
    """
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Cat', 'Dog'], yticklabels=['Cat', 'Dog'])
    plt.title(f'{model_name} - Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

def visualize_predictions(model, validation_generator, num_images=8):
    """
    Visualize model predictions on sample images.
    This helps understand what the model has learned and where it makes mistakes.

    Args:
        model: Trained model
        validation_generator: Validation data generator
        num_images: Number of images to display
    """
    # Get a batch of images
    validation_generator.reset()
    batch = next(validation_generator)
    images, true_labels = batch

    # Make predictions
    predictions = model.predict(images[:num_images])

    # Plot images with predictions
    plt.figure(figsize=(15, 8))
    for i in range(num_images):
        plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])

        # Get predicted and true labels
        pred_prob = predictions[i][0]
        pred_label = 'Dog' if pred_prob > 0.5 else 'Cat'
        true_label = 'Dog' if true_labels[i] == 1 else 'Cat'

        # Color: green if correct, red if incorrect
        color = 'green' if pred_label == true_label else 'red'

        plt.title(f'True: {true_label}\nPred: {pred_label} ({pred_prob:.2f})',
                 color=color, fontsize=10)
        plt.axis('off')

    plt.suptitle(f'{model.name} - Sample Predictions', fontsize=14)
    plt.tight_layout()
    plt.show()



In [ ]:
# ============================================================================
# PART 4: MAIN EXECUTION
# ============================================================================

def main():
    """
    Main function to run the complete CNN experiment.
    """
    print("CATS VS DOGS CLASSIFICATION WITH CNN")
    print("Working in current directory to avoid permission issues")
    print("=" * 60)

    # Download and prepare dataset
    train_dir, validation_dir = download_cats_dogs_dataset()

    # Create data generators
    train_generator, validation_generator = create_data_generators(
        train_dir, validation_dir,
        img_height=150, img_width=150,
        batch_size=32
    )

    # Visualize sample images
    print("\nVisualizing sample training images...")
    visualize_sample_images(train_generator)

    # Create and train simple CNN
    print(f"\n{'='*60}")
    print("TRAINING SIMPLE CNN")
    print(f"{'='*60}")

    simple_cnn = create_simple_cnn(150, 150)
    history_simple = train_model(simple_cnn, train_generator, validation_generator, epochs=8)

    # Evaluate simple CNN
    acc_simple, pred_simple, true_simple = evaluate_model(simple_cnn, validation_generator)

    # Visualize training history
    plot_training_history(history_simple, "Simple CNN")
    plot_confusion_matrix(true_simple, pred_simple, "Simple CNN")
    visualize_predictions(simple_cnn, validation_generator)

    # Create and train improved CNN
    print(f"\n{'='*60}")
    print("TRAINING IMPROVED CNN")
    print(f"{'='*60}")

    improved_cnn = create_improved_cnn(150, 150)
    history_improved = train_model(improved_cnn, train_generator, validation_generator, epochs=8)

    # Evaluate improved CNN
    acc_improved, pred_improved, true_improved = evaluate_model(improved_cnn, validation_generator)

    # Visualize training history
    plot_training_history(history_improved, "Improved CNN")
    plot_confusion_matrix(true_improved, pred_improved, "Improved CNN")
    visualize_predictions(improved_cnn, validation_generator)

    # Final comparison
    print(f"\n{'='*60}")
    print("FINAL COMPARISON")
    print(f"{'='*60}")
    print(f"Simple CNN Accuracy:   {acc_simple:.4f} ({acc_simple*100:.2f}%)")
    print(f"Improved CNN Accuracy: {acc_improved:.4f} ({acc_improved*100:.2f}%)")

    if acc_improved > acc_simple:
        improvement = ((acc_improved - acc_simple) / acc_simple) * 100
        print(f"Improvement: {improvement:.2f}%")

    # Model parameters comparison
    print(f"\nModel Complexity Comparison:")
    print(f"Simple CNN Parameters:   {simple_cnn.count_params():,}")
    print(f"Improved CNN Parameters: {improved_cnn.count_params():,}")

    # Save models
    simple_cnn.save("simple_cnn_model.h5")
    improved_cnn.save("improved_cnn_model.h5")
    print("\nModels saved to current directory!")

if __name__ == "__main__":
    main()



## DISCUSSION QUESTIONS FOR STUDENTS

1. Why are CNNs better suited for image classification compared to regular dense networks?
   Hint: Think about spatial relationships and parameter sharing

2. What is the purpose of each layer type:
   - Convolutional layers: Feature extraction with learnable filters
   - Pooling layers: Dimensionality reduction and translation invariance
   - Dropout layers: Regularization to prevent overfitting
   - Batch normalization: Stabilize training and improve convergence

3. How does data augmentation help improve model performance?
   Hint: Think about dataset size and model generalization

4. What happens to the spatial dimensions as data flows through the CNN?
   Hint: Track the output shape after each layer

5. Why do we use different numbers of filters in different layers?
   Hint: Think about feature hierarchy (simple → complex)

6. How do early stopping and learning rate reduction help during training?
   Hint: Think about overfitting and optimization

7. Experiment: Try different architectures. What happens if you:
   - Remove pooling layers?
   - Use different filter sizes (5x5 instead of 3x3)?
   - Add more/fewer layers?
   - Change the number of filters?

8. What are the trade-offs between model complexity and performance?
   Hint: Consider training time, memory usage, and accuracy

TROUBLESHOOTING:
- If download fails, the code will create synthetic demo images
- All files are saved to current directory (no permission issues)
- Required packages: pip install tensorflow pillow scikit-learn matplotlib seaborn

EXPECTED RESULTS:
- Simple CNN: ~75-85% accuracy
- Improved CNN: ~85-95% accuracy  
- Training should complete in 5-15 minutes depending on hardware
